In [ ]:
import pandas as pd
from reldi.ner_tagger import NERTagger
import json

n=NERTagger('hr')
n.authorize('datamilas','Jasamana1')


In [24]:
#read data from sraping
df = pd.read_csv('data.csv')



In [26]:
#define which data to analyze

#by song_ID
df["toAnalyze"] = df["Song_ID"].str.find('Zwn2Yv0K') 
#df_old = df[df.toAnalyze == -1]
#df_old = df_old.drop(["toAnalyze"], axis=1)

df = df[df.toAnalyze >= 0]
df = df.reset_index()
df = df.drop(["index", "toAnalyze"], axis=1)


In [5]:
df_tagged = pd.DataFrame()

df_tagged_lyrics = pd.DataFrame()

In [6]:
songs = df["Song"]
lyrics = df["Lyrics"]

for index, song in enumerate(songs):
    tagged = json.loads(n.tag(song))
    df_tagged = df_tagged.append(tagged, ignore_index=True)
    
    text = lyrics[index]
    tagged_text = json.loads(n.tag(text))
    df_tagged_lyrics = df_tagged_lyrics.append(tagged_text, ignore_index=True)

In [8]:
column_names = ["Artist", "Song", "Song_ID", "Surface", "Tags", "Lemma", "Entity"]
output_data = pd.DataFrame(columns = column_names)

for index, row in df.iterrows():
    for counter, word in enumerate(df_tagged["tokens"][index]["token"]):
        new_entry = pd.DataFrame([[
        row["Artist"],
        row["Song"],
        row["Song_ID"],
        word["text"],
        df_tagged["POStags"][index]["tag"][counter]["text"],
        df_tagged["lemmas"][index]["lemma"][counter]["text"],
        df_tagged["namedEntities"][index]["entity"][counter]["value"]]], 
        columns = column_names)

        output_data = output_data.append(new_entry, ignore_index=True)


In [10]:
column_names = ["Artist", "Song", "Song_ID", "Surface", "Tags", "Lemma", "Entity"]
output_data_lyrics = pd.DataFrame(columns = column_names)

for index, row in df.iterrows():
    for counter, word in enumerate(df_tagged_lyrics["tokens"][index]["token"]):
        new_entry = pd.DataFrame([[
        row["Artist"],
        row["Song"],
        row["Song_ID"],
        word["text"],
        df_tagged_lyrics["POStags"][index]["tag"][counter]["text"],
        df_tagged_lyrics["lemmas"][index]["lemma"][counter]["text"],
        df_tagged_lyrics["namedEntities"][index]["entity"][counter]["value"]]], 
        columns = column_names)

        output_data_lyrics = output_data_lyrics.append(new_entry, ignore_index=True)

In [ ]:
#append to existing data

output_data.to_csv('titles.csv', index=False, encoding='utf-8', mode='a', header=False)
output_data_lyrics.to_csv('lyrics.csv', index=False, encoding='utf-8', mode='a', header=False)